# お薦め商品の提案（Association分析）

## 処理 Pattern と想定される業務利用 Sean
### Association 分析
- 大量の商品購入履歴 Data から「商品Ａと商品Ｂが同時に購入されることが多い」といった法則を見つけ出す分析手法。
- 「商品Ａと商品Ｂを Set にして割り引き価格を設定する」といった Marketing施策を導く
- 単価が少額の商品Ｘの仕入れを継続するか検討する時、分析の結果、高単価の商品Ｙと Set で売れており売上に貢献していることが判明。商品Ｘの仕入れを継続する判断を導く

## 教師なし学習（分析）の処理 Pattern
単に分析結果を出しただけでは、AI の効果がでたことにならない。
**得られた知見を基に、何か戦略を考え、その効果があって初めて分析の意味を持つ**

### 教師あり学習との違い
| 学習方法   | 特徴                                                                                           |
|--------|----------------------------------------------------------------------------------------------|
| 教師あり学習 | 正解 Data の収集が大変だが、Model構築までできれば業務での活用は比較的簡単なことが多い。                                            |
| 教師なし学習 | 正解 Data の収集は不要なので Model構築は比較的簡単。しかし、業務での活用は「知見を基に戦略を考え、それが効果を出す」と1 Action 余分に必要なので比較的難易度が高い。 |

## 例題の Data 説明と Use case

Data: UCI Dataset の「Online Retail Data Set 」（Online 小売り Data Set）を利用。
イギリスを拠点とし、Gift 商品を中心に販売している EC Site の取引履歴 Data
- 取引はまとまった単位で行なわれているので、一般顧客より小売業者が主な対象と考えられる。
- Data 件数は総数で 54万件
- 商品購買分析の例題でよく利用される Data.

この Data からフランスの顧客からよく Set 購入されていることが商品を見つけだす。